In [ ]:
import Simulations
from Algo_PLL import PhaseTracker as PLL
from Algo_TWave import PhaseTracker as TWave
from Algo_AmpTh import PhaseTracker as AmpTh
from Algo_SineFit import PhaseTracker as SineFit
from Algo_ZeroCrossing import PhaseTracker as ZeroCross

from Inhibitors import *
from tqdm import tqdm

import os
from concurrent.futures import ProcessPoolExecutor


In [ ]:
def process_subject(rundata):
    subject, algo_name = rundata

    # load the dataset
    ds = Simulations.load_anphy_data(subject=subject)

    # prepare the algorithm
    if algo_name == 'PLL':
        algo = PLL(
            fs=ds.fs,
            inhibitors=[
                MinAmp(window_length_sp=int(ds.fs * 2),
                    min_amp_threshold_uv=45),
            ],
        )
    elif algo_name == 'TWave':
        algo = TWave(ds.fs)
    elif algo_name == 'AmpTh':
        algo = AmpTh(
            stim_delay_sp=int(ds.fs * 0.5),
            adaptive_window_sp=int(ds.fs * 5),
            backoff_sp=int(ds.fs * 5),
        )
    elif algo_name == 'SineFit':
        algo = SineFit(
        )
    elif algo_name == 'ZeroCrossing':
        algo = ZeroCross(
        )

    # run the simulation
    rslt = Simulations.run_simulations(ds, algo)

    return rslt


# for each subject in the anphy dataset, load Fpz-Cz, run the phase tracker algorithm
subjects = Simulations.load_anphy_data()
algo_names = ['PLL', 'TWave', 'AmpTh', 'SineFit', 'ZeroCrossing']

# create a list of tuples (subject, algo_name)
rundata = [(subject, algo_name)
           for subject in subjects for algo_name in algo_names]

with ProcessPoolExecutor(max_workers=os.cpu_count() - 2) as executor:
    # executor.map returns results in order; wrap it with tqdm to show progress
    results = list(
        tqdm(executor.map(process_subject, rundata), total=len(rundata)))

# separate out each algo
results_by_algo = {}
for algo_name in algo_names:
    results_by_algo[algo_name] = list(filter(lambda x: x.PhaseTracker.name == algo_name, results))

# do group average
group_rslt = {}
for algo_name, algo_results in results_by_algo.items():
    # create a group result object
    group_rslt[algo_name] = Simulations.SimulationGroupResult(algo_results, name='anphy')


In [ ]:
for algo_name, this_group_rslt in group_rslt.items():
    stim_phase = this_group_rslt.compute_stim_phase()
    np.savetxt(
        f"output/{this_group_rslt.name}_{this_group_rslt.phase_tracker_name}_stim_phase.csv",
        stim_phase,
        delimiter=",")

    hf = this_group_rslt.plot_phase_hist()
    hf.savefig(
        f"output/{this_group_rslt.name}_{this_group_rslt.phase_tracker_name}_phase_hist.pdf",
        bbox_inches="tight",
        transparent=True)

    hf = this_group_rslt.plot_evoked()
    hf.savefig(
        f"output/{this_group_rslt.name}_{this_group_rslt.phase_tracker_name}_evoked.pdf",
        bbox_inches="tight",
        transparent=True)

In [ ]:
hf = this_group_rslt.plot_evoked()
hf.savefig(
    f"output/{this_group_rslt.name}_{this_group_rslt.phase_tracker_name}_evoked.pdf",
    bbox_inches="tight",
    transparent=True)